<a href="https://colab.research.google.com/github/AbdulxoliqMirzayev/Toshkent_shahar_uylar_narxi_ML/blob/main/Tashkent_price_houses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Toshkent shaxridagi uylarning narxini bashorat qilish......

import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/anvarnarz/praktikum_datasets/main/housing_data_08-02-2021.csv')
df.head()

,location,district,rooms,size,level,max_levels,price
0,"город Ташкент, Юнусабадский район, Юнусабад 8-...",Юнусабадский,3,57,4,4,52000
1,"город Ташкент, Яккасарайский район, 1-й тупик ...",Яккасарайский,2,52,4,5,56000
2,"город Ташкент, Чиланзарский район, Чиланзар 2-...",Чиланзарский,2,42,4,4,37000
3,"город Ташкент, Чиланзарский район, Чиланзар 9-...",Чиланзарский,3,65,1,4,49500
4,"город Ташкент, Чиланзарский район, площадь Актепа",Чиланзарский,3,70,3,5,55000


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# 1. Ma'lumotlarni yuklash
df = pd.read_csv('https://raw.githubusercontent.com/anvarnarz/praktikum_datasets/main/housing_data_08-02-2021.csv')

# 2. Narx ustunini tozalash
df = df[df['price'].apply(lambda x: str(x).replace('.', '', 1).isdigit())]
df['price'] = df['price'].astype(float)

# 3. Size ustunini tozalash
df['size'] = df['size'].str.replace('Площадьземли:', '').str.replace('сот', '').str.replace(' ', '').astype(float)

# 4. Null qiymatlarni to'ldirish
for column in df.select_dtypes(include=['float64', 'int64']):
    df[column].fillna(df[column].median(), inplace=True)

# 5. Kategoriyali ustunlarni OneHotEncoding qilish
df = pd.get_dummies(df, columns=['district', 'location'], drop_first=True)

# 6. O'qitish va test ma'lumotlarini ajratish
X = df.drop('price', axis=1)
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 7. Raqamli ma'lumotlarni standartizatsiya qilish
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 8. Random Forest modelini yaratish va o'qitish
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)

# 9. Linear Regression modelini yaratish va o'qitish
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

# 10. Bashorat qilish
rf_y_pred = rf_model.predict(X_test_scaled)
lr_y_pred = lr_model.predict(X_test_scaled)

# 11. Xatolikni hisoblash
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_y_pred))
lr_rmse = np.sqrt(mean_squared_error(y_test, lr_y_pred))

print(f'Random Forest RMSE: {rf_rmse:.2f}')
print(f'Linear Regression RMSE: {lr_rmse:.2f}')

# 12. Kross-valyatsiya
rf_cv_scores = cross_val_score(rf_model, X_train_scaled, y_train, scoring="neg_mean_squared_error", cv=10)
lr_cv_scores = cross_val_score(lr_model, X_train_scaled, y_train, scoring="neg_mean_squared_error", cv=10)

# RMSE hisoblash
rf_rmse_cv = np.sqrt(-rf_cv_scores)
lr_rmse_cv = np.sqrt(-lr_cv_scores)

# Natijalarni chiqarish
print("\nRandom Forest Cross-Validation RMSE:")
print("Scores:", rf_rmse_cv)
print("Mean:", rf_rmse_cv.mean())
print("Standard deviation:", rf_rmse_cv.std())

print("\nLinear Regression Cross-Validation RMSE:")
print("Scores:", lr_rmse_cv)
print("Mean:", lr_rmse_cv.mean())
print("Standard deviation:", lr_rmse_cv.std())


Random Forest RMSE: 1367007.95
Linear Regression RMSE: 1135946072847818368.00

Random Forest Cross-Validation RMSE:
Scores: [269580.70179943  45780.40609197  22613.08908275 155238.08930167
  20571.00188295  57891.75873604 207457.02998482  75295.75690427
 630350.77507745 174254.76006825]
Mean: 165903.33689296036
Standard deviation: 174450.55396972646

Linear Regression Cross-Validation RMSE:
Scores: [2.97284834e+19 1.75367283e+19 3.98393748e+19 3.46173904e+19
 4.12099875e+19 6.62219415e+19 5.06935691e+19 5.84064004e+19
 2.32642088e+19 3.23373548e+19]
Mean: 3.9385543902357545e+19
Standard deviation: 1.4547613107232414e+19
